In [ ]:
import pandas as pd
import numpy as np
import io
import zipfile
from tqdm import tqdm
import missingno as msno

!pip install pykrx
from pykrx import stock
!pip install finance-datareader --upgrade
import FinanceDataReader as fdr
import time
# !pip install zipline
# from zipline.pipeline.factors import Returns

import requests
import json
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ETREE
from typing import *
import statsmodels.api as sm

# from hmmlearn.hmm import GaussianHMM
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# 코랩에 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")

### 1.재무제표 계정과목 데이터

* 재무상태표(BS)

  매출채권, 매입채무, 단기차입금, 장기매입채무 및 기타비유동채무, 장기매출채권 및 기타비유동채권

* 포괄손익계산서

  영업이익 (변화율 : (FY2, 1m), (FQ1, 1m), (FQ0, yoy)) 당기순이익 (증가율),

  매출, 총포괄손익, 매출원가, 매출총이익, 영업이익, 당기순이익(손실), 판매비와 관리비[ 판매비용과 관리비용], 법인세비용, 주당이익[주당손실], 대손상각비(손실충당금환입), 대손충당금

* 현금흐름표

  감가상각비

In [ ]:
BS_account_subject_list = ["ifrs-full_Assets", "ifrs-full_Liabilities", "ifrs-full_Equity",
                           "dart_ShortTermTradeReceivable", "dart_ShortTermTradePayables",
                           "ifrs-full_CurrentAssets", "ifrs-full_CurrentLiabilities", "ifrs-full_CashAndCashEquivalents",
                           "dart_LongTermBorrowingsGross", "ifrs-full_ShorttermBorrowings"]
# 자산 | 부채 | 자본 | 매출채권 | 매입채무 | 유동자산 | 유동부채 | 현금성자산 | 장기차입금 | 총차입금 | 감가상각비

In [ ]:
CIS_account_subject_list = ["ifrs-full_Revenue", "ifrs-full_CostOfSales", "ifrs-full_GrossProfit", "dart_TotalSellingGeneralAdministrativeExpenses", "dart_OperatingIncomeLoss", "ifrs-full_ProfitLossBeforeTax", "ifrs-full_ProfitLoss", "ifrs-full_EarningsPerShareAbstract"]
# 매출액 | 매출원가 | 매출총이익 | 판매비와관리비 | 영업이익(손실) | 법인세비용차감전순이익(손실)	| 당기순이익(손실) | 주당손익(EPS)

In [ ]:
CF_account_subject_list = ["ifrs-full_CashFlowsFromUsedInOperatingActivities", "dart_AdjustmentsForDepreciationExpense", "dart_AdjustmentsForInterestIncome", "dart_AdjustmentsForInterestExpenses",
                           "ifrs-full_AdjustmentsForIncomeTaxExpense", "ifrs-full_CashFlowsFromUsedInInvestingActivities", "ifrs-full_CashFlowsFromUsedInFinancingActivities"]
# 영업활동현금흐름 | 감가상각비 | 이자수익 | 이자비용 | 법인세비용(수익) | 투자활동현금흐름 | 재무활동현금흐름

In [ ]:
class HandlingData():

  def __init__(self,
               bs_path: str,
               cis_path: str,
               cf_path: str):
    self.BalanceSheet = pd.read_csv(
        bs_path, index_col = False, encoding = "cp949", sep = "\t")
    self.ComprehensiveIncomeSheet = pd.read_csv(
        cis_path, index_col = False, encoding = "cp949", sep = "\t")
    self.CashFlowSheet = pd.read_csv(
        cf_path, index_col = False, encoding = "cp949", sep = "\t")


  def dataInformation(self, stock_code_name: str):
    for data in [self.BalanceSheet,  self.ComprehensiveIncomeSheet, self.CashFlowSheet]:
      data[stock_code_name] = data[stock_code_name].apply( lambda x: str(x).replace("[", "") )
      data[stock_code_name] = data[stock_code_name].apply( lambda x: str(x).replace("]", "") )
      display(data.info())


  def dataPreparation(self, stock_code_name: str, account_subject_name: str,
                      bs_base_date: str, cis_base_date: str, cf_base_date: str):
    self.BalanceSheet = pd.pivot_table(
        data = self.BalanceSheet, index = stock_code_name, columns = account_subject_name, values = bs_base_date, aggfunc = "sum")
    self.ComprehensiveIncomeSheet = pd.pivot_table(
        data = self.ComprehensiveIncomeSheet, index = stock_code_name, columns = account_subject_name, values = cis_base_date, aggfunc = "sum")
    self.CashFlowSheet = pd.pivot_table(
        data = self.CashFlowSheet, index = stock_code_name, columns = account_subject_name, values = cf_base_date, aggfunc = "sum")
    return self.BalanceSheet,  self.ComprehensiveIncomeSheet, self.CashFlowSheet


  def accountSubjectSelection(self, bs_subject: List, cis_subject: List, cf_subject: List):
    self.BalanceSheet = self.BalanceSheet[bs_subject]
    self.ComprehensiveIncomeSheet = self.ComprehensiveIncomeSheet[cis_subject]
    self.CashFlowSheet = self.CashFlowSheet[cf_subject]

    FinancialSheet = pd.concat([self.BalanceSheet, self.ComprehensiveIncomeSheet], ignore_index = False, axis = 1)
    FinancialSheet = pd.concat([FinancialSheet, self.CashFlowSheet], ignore_index = False, axis = 1)
    return FinancialSheet


  # Step 2: Factor Selection
  # Step 3: Factor Calculation: Calculate factor values for each security
  # Step 4: Rank Securities: Rank securities based on each factor
  # Step 5: Factor Weighting: Assign weights to each factor

```
    pd.set_option("display.max_row", 50)
    CIS_2023_1Q_ = pd.pivot_table(
        data = CIS_2023_1Q,
        index = "종목코드",
          # 각 행(row)는 무엇으로 정의할지
        columns = "항목코드",
          # 각 열(column)은 무엇으로 정의할지
        values = "당기 1분기 3개월",
          # 각 Cell을 어떤 숫자로 계산할지
        aggfunc = "sum"
            # 계산을 어떻게 할지 : 'mean', 'sum', 'nunique' 등
          # fill_value = None,
          # margins = False,
          # dropna = True,
          # margins_name = "All",
          # observed = False,
          # sort = True
    )
```

* 2022년 1분기

In [ ]:
FS20221Q = HandlingData(
  bs_path = "/content/drive/MyDrive/FS_DATA/20221Q_BS.txt",
  cis_path = "/content/drive/MyDrive/FS_DATA/20221Q_CIS.txt",
  cf_path = "/content/drive/MyDrive/FS_DATA/20221Q_CF.txt"
)

In [ ]:
FS20221Q.dataInformation(stock_code_name = "종목코드")

In [ ]:
BS_2022_1Q, CIS_2022_1Q, CF_2022_1Q = FS20221Q.dataPreparation(stock_code_name = "회사명", account_subject_name = "항목코드",
                          bs_base_date = "당기 1분기말", cis_base_date = "당기 1분기 3개월", cf_base_date = "당기 1분기")

In [ ]:
FS_2022_1Q = FS20221Q.accountSubjectSelection(
    bs_subject = BS_account_subject_list,
    cis_subject = CIS_account_subject_list,
    cf_subject = CF_account_subject_list)

In [ ]:
FS_2022_1Q

In [ ]:
FS_2022_1Q.to_csv("/content/drive/MyDrive/FindAlpha_RiskyAsset1/data/FinancialSheetData/FS_2022_1Q.csv", index = True)

* 2022년 2분기

In [ ]:
FS20222Q = HandlingData(
  bs_path = "/content/drive/MyDrive/FS_DATA/20222Q_BS.txt",
  cis_path = "/content/drive/MyDrive/FS_DATA/20222Q_CIS.txt",
  cf_path = "/content/drive/MyDrive/FS_DATA/20222Q_CF.txt"
)

In [ ]:
FS20222Q.dataInformation(stock_code_name = "종목코드")

In [ ]:
BS_2022_2Q, CIS_2022_2Q, CF_2022_2Q = FS20222Q.dataPreparation(stock_code_name = "회사명", account_subject_name = "항목코드",
                         bs_base_date = "당기 반기말", cis_base_date = "당기 반기 3개월", cf_base_date = "당기 반기")

In [ ]:
FS_2022_2Q = FS20222Q.accountSubjectSelection(
    bs_subject = BS_account_subject_list,
    cis_subject = CIS_account_subject_list,
    cf_subject = CF_account_subject_list)

In [ ]:
FS_2022_2Q.to_csv("/content/drive/MyDrive/FindAlpha_RiskyAsset1/data/FinancialSheetData/FS_2022_2Q.csv", index = True)

* 2022년 3분기

In [ ]:
FS20223Q = HandlingData(
  bs_path = "/content/drive/MyDrive/FS_DATA/20223Q_BS.txt",
  cis_path = "/content/drive/MyDrive/FS_DATA/20223Q_CIS.txt",
  cf_path = "/content/drive/MyDrive/FS_DATA/20223Q_CF.txt"
)

In [ ]:
FS20223Q.dataInformation(stock_code_name = "종목코드")

In [ ]:
BS_2022_3Q, CIS_2022_3Q, CF_2022_3Q = FS20223Q.dataPreparation(stock_code_name = "회사명", account_subject_name = "항목코드",
                         bs_base_date = "당기 3분기말", cis_base_date = "당기 3분기 3개월", cf_base_date = "당기 3분기")

In [ ]:
FS_2022_3Q = FS20223Q.accountSubjectSelection(
    bs_subject = BS_account_subject_list,
    cis_subject = CIS_account_subject_list,
    cf_subject = CF_account_subject_list)

In [ ]:
FS_2022_3Q.to_csv("/content/drive/MyDrive/FindAlpha_RiskyAsset1/data/FinancialSheetData/FS_2022_3Q.csv", index = True)

* 2022년 4분기

In [ ]:
FS20224Q = HandlingData(
  bs_path = "/content/drive/MyDrive/FS_DATA/20224Q_BS.txt",
  cis_path = "/content/drive/MyDrive/FS_DATA/20224Q_CIS.txt",
  cf_path = "/content/drive/MyDrive/FS_DATA/20224Q_CF.txt"
)

In [ ]:
FS20224Q.dataInformation(stock_code_name = "종목코드")

In [ ]:
BS_2022_4Q, CIS_2022_4Q, CF_2022_4Q = FS20224Q.dataPreparation(stock_code_name = "회사명", account_subject_name = "항목코드",
                         bs_base_date = "당기", cis_base_date = "당기", cf_base_date = "당기")

In [ ]:
FS_2022_4Q = FS20224Q.accountSubjectSelection(
    bs_subject = BS_account_subject_list,
    cis_subject = CIS_account_subject_list,
    cf_subject = CF_account_subject_list)

In [ ]:
FS_2022_4Q.to_csv("/content/drive/MyDrive/FindAlpha_RiskyAsset1/data/FinancialSheetData/FS_2022_4Q.csv", index = True)

* 2023년 1분기

In [ ]:
FS20231Q = HandlingData(
  bs_path = "/content/drive/MyDrive/FS_DATA/20231Q_BS.txt",
  cis_path = "/content/drive/MyDrive/FS_DATA/20231Q_CIS.txt",
  cf_path = "/content/drive/MyDrive/FS_DATA/20231Q_CF.txt"
)

In [ ]:
FS20231Q.dataInformation(stock_code_name = "종목코드")

In [ ]:
BS_2023_1Q, CIS_2023_1Q, CF_2022_3Q = FS20231Q.dataPreparation(stock_code_name = "회사명", account_subject_name = "항목코드",
                          bs_base_date = "당기 1분기말", cis_base_date = "당기 1분기 3개월", cf_base_date = "당기 1분기")

In [ ]:
FS_2023_1Q = FS20231Q.accountSubjectSelection(
    bs_subject = BS_account_subject_list,
    cis_subject = CIS_account_subject_list,
    cf_subject = CF_account_subject_list)

In [ ]:
FS_2023_1Q.to_csv("/content/drive/MyDrive/FindAlpha_RiskyAsset1/data/FinancialSheetData/FS_2023_1Q.csv", index = True)

___

### 2.멀티팩터 포트폴리오를 구성하는 방법

* Mixed 방식

  각 팩터에 대한 투자 포트폴리오를 비중을 달리하여 투자하는 것

* Integrated 방식

  투자 유니버스의 각 종목에 대하여 원하는 팩터에 노출되어 있는 정도에 따라 순위를 매겨 이를 기반으로 점수를 계산한 후, z-score나 rank를 활용하여 합산 점수에 의한 포트폴리오를 구성하는 것

* Sequential 방식(Screening 방식)

  밸류 팩터 상위 포트폴리오에 속하는 종목들을 다시 모멘텀 팩터로 정렬하여 밸류 팩터 상위 종목 중 모멘텀 상위 종목을 고르거나, 또는 순서를 반대로 하는 방법


### 팩터 유형

* 퀄리티 팩터 : 회사의 과거와 현재 영업 경쟁력이 얼마나 주가에 반영되고 있는지를 보는 지표(재무구조와 수익성이 뛰어난 종목)

* 모멘텀 팩터 : 회사의 실적 추이나 과거의 주가 상승 트렌드가 주가에 반영되는지를 보는 팩터(최근 상승세를 기록한 종목)

* 밸류에이션 팩터 : 특정 밸류에이션 레벨에서 투자하였을 때 좋은 투자 성과가 나타나는지를 보는 팩터

In [61]:
class MultiFactorPortfolio():

  def __init__(self,
               path_2022_1Q: str, path_2022_2Q: str, path_2022_3Q: str, path_2022_4Q: str, path_2023_1Q: str,
               path_KP_close: str, path_KP_volume: str, path_KDQ_close: str, path_KDQ_volume: str,
               KOR_SECTOR = None, KOSPI_TICKER = None, KOSDAQ_TICKER = None):

    self.FS_2022_1Q = pd.read_csv(path_2022_1Q, index_col = 0)
    self.FS_2022_2Q = pd.read_csv(path_2022_2Q, index_col = 0)
    self.FS_2022_3Q = pd.read_csv(path_2022_3Q, index_col = 0)
    self.FS_2022_4Q = pd.read_csv(path_2022_4Q, index_col = 0)
    self.FS_2023_1Q = pd.read_csv(path_2023_1Q, index_col = 0)

    self.KOSPI_CLOSE = pd.read_csv(path_KP_close, index_col = False)
    self.KOSPI_VOLUME = pd.read_csv(path_KP_volume, index_col = False)
    self.KOSDAQ_CLOSE = pd.read_csv(path_KDQ_close, index_col = False)
    self.KOSDAQ_VOLUME = pd.read_csv(path_KDQ_volume, index_col = False)

    self.KOR_SECTOR = KOR_SECTOR
    self.KOSPI_TICKER = KOSPI_TICKER
    self.KOSDAQ_TICKER = KOSDAQ_TICKER


  def loadFinancialSheet(self):
    for data in [self.FS_2022_1Q, self.FS_2022_2Q, self.FS_2022_3Q, self.FS_2022_4Q, self.FS_2023_1Q]:
      data.columns = [
          "자산", "부채", "자본", "매출채권", "매입채무", "유동자산", "유동부채", "현금성자산", "장기차입금", "총차입금",
          "매출액", "매출원가", "매출총이익", "판매비와관리비", "영업이익(손실)", "법인세비용차감전순이익(손실)", "당기순이익(손실)", "주당손익(EPS)",
          "영업활동현금흐름", "감가상각비", "이자수익", "이자비용", "법인세비용(수익)", "투자활동현금흐름", "재무활동현금흐름"]
    return self.FS_2022_1Q, self.FS_2022_2Q, self.FS_2022_3Q, self.FS_2022_4Q, self.FS_2023_1Q


  def getStockCode(self, date, market):
    stock_list = pd.DataFrame(
        {"종목코드": stock.get_market_ticker_list(date, market = market)})

    stock_list["종목명"] = stock_list["종목코드"].map(lambda x: stock.get_market_ticker_name(x))
    # https://github.com/financedata-org/FinanceDataReader/blob/develop/krx/data.py
    # fdr_data = fdr.StockListing("KRX")
    # stock_list["업종"] = stock_list["종목명"].map(lambda x: fdr_data[fdr_data["Name"] == x]["Sector"].iloc[0])
    return stock_list


  def getStockList(self, base_date):
    self.KOSPI_TICKER = self.getStockCode(date = base_date, market = "KOSPI")
    self.KOSDAQ_TICKER = self.getStockCode(date = base_date, market = "KOSDAQ")


  def getStockSector(self, sector_code_list: List):
    DATA_SECTOR: List = []
    for sector_code in sector_code_list:
        URL = f'''https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20230712&sec_cd={ sector_code }'''
        DATA = requests.get(URL).json()
        DATA_JSON = pd.json_normalize(DATA["list"])
        DATA_SECTOR.append(DATA_JSON)
        time.sleep(2)
    KOR_SECTOR = pd.concat(DATA_SECTOR, axis = 0)
    KOR_SECTOR.rename(columns = {"CMP_CD": "TICKER"}, inplace = True)
    self.KOR_SECTOR = KOR_SECTOR[["TICKER", "SEC_NM_KOR"]]

    # sector_code_list = ["G10","G15","G20","G25", "G30", "G35", "G40", "G45","G50", "G55"]


  def getFundamental(self, base_date: str):
    KOSPI_FUNDAMENTAL = stock.get_market_fundamental_by_ticker(
        date = base_date, market = "KOSPI")
    KOSPI_FUNDAMENTAL["종목코드"] = KOSPI_FUNDAMENTAL.index
    KOSPI_FUNDAMENTAL.reset_index(drop = True, inplace = True)
    self.KOSPI_TICKER = pd.merge(
        self.KOSPI_TICKER, KOSPI_FUNDAMENTAL,
        left_on = "종목코드", right_on = "종목코드", how = "outer")
    self.KOSPI_TICKER = pd.merge(
        self.KOSPI_TICKER, self.KOR_SECTOR,
        left_on = "종목코드", right_on = "TICKER", how = "left")

    KOSDAQ_FUNDAMENTAL = stock.get_market_fundamental_by_ticker(
        date = base_date, market = "KOSDAQ")
    KOSDAQ_FUNDAMENTAL["종목코드"] = KOSDAQ_FUNDAMENTAL.index
    KOSDAQ_FUNDAMENTAL.reset_index(drop = True, inplace = True)
    self.KOSDAQ_TICKER = pd.merge(
        self.KOSDAQ_TICKER, KOSDAQ_FUNDAMENTAL,
        left_on = "종목코드", right_on = "종목코드", how = "outer")
    self.KOSDAQ_TICKER = pd.merge(
        self.KOSDAQ_TICKER, self.KOR_SECTOR,
        left_on = "종목코드", right_on = "TICKER", how = "left")

    return self.KOSPI_TICKER, self.KOSDAQ_TICKER


# getKOSPIMarketPrice()와 getKOSDAQMarketPrice()는 1회 실행하여 데이터 저장 후 로드하여 활용할 것
  def getKOSPIMarketPrice(self):
    i: int = 0
    KOSPI_CLOSE: List = []
    KOSPI_CLOSE_DATA = pd.DataFrame()
    KOSPI_VOLUME: List = []
    KOSPI_VOLUME_DATA = pd.DataFrame()

    for ticker in tqdm(self.KOSPI_TICKER["종목코드"].tolist()):
      KOSPI_CLOSE.append(fdr.DataReader(ticker, "2022")["Close"])
      KOSPI_VOLUME.append(fdr.DataReader(ticker, "2022")["Volume"])
      KOSPI_CLOSE_DATA[ticker] = KOSPI_CLOSE[i]
      KOSPI_VOLUME_DATA[ticker] = KOSPI_CLOSE[i]
        # ValueError: Cannot set a DataFrame with multiple columns to the single column 095570
      i += 1
    # KOSPI_CLOSE_DATA.to_csv("/content/drive/MyDrive/FSDATA/KOSPI_CLOSE.csv", index = True)
    # KOSPI_VOLUME_DATA.to_csv("/content/drive/MyDrive/FSDATA/KOSPI_VOLUME.csv", index = True)
    self.KOSPI_CLOSE_DATA = KOSPI_CLOSE_DATA
    self.KOSPI_VOLUME_DATA = KOSPI_VOLUME_DATA


  def getKOSDAQMarketPrice(self):
    i: int = 0
    KOSDAQ_CLOSE: List = []
    KOSDAQ_CLOSE_DATA = pd.DataFrame()
    KOSDAQ_VOLUME: List = []
    KOSDAQ_VOLUME_DATA = pd.DataFrame()

    for ticker in tqdm(self.KOSDAQ_TICKER["종목코드"].tolist()):
      KOSDAQ_CLOSE.append(fdr.DataReader(ticker, "2022")["Close"])
      KOSDAQ_VOLUME.append(fdr.DataReader(ticker, "2022")["Volume"])
      KOSDAQ_CLOSE_DATA[ticker] = KOSDAQ_CLOSE[i]
      KOSDAQ_VOLUME_DATA[ticker] = KOSDAQ_CLOSE[i]
        # ValueError: Cannot set a DataFrame with multiple columns to the single column 095570
      i += 1
    # KOSDAQ_CLOSE_DATA.to_csv("/content/drive/MyDrive/FSDATA/KOSDAQ_CLOSE.csv", index = True)
    # KOSDAQ_VOLUME_DATA.to_csv("/content/drive/MyDrive/FSDATA/KOSDAQ_VOLUME.csv", index = True)
    self.KOSDAQ_CLOSE_DATA = KOSDAQ_CLOSE_DATA
    self.KOSDAQ_VOLUME_DATA = KOSDAQ_VOLUME_DATA


  def getMarketPQ(self, start_period: str, end_period: str):

    for data in [self.KOSPI_CLOSE, self.KOSPI_VOLUME, self.KOSDAQ_CLOSE, self.KOSDAQ_VOLUME]:
      data["Date"] = pd.to_datetime(data["Date"])
      data = data.loc[start_period:end_period, :]
    return self.KOSPI_CLOSE, self.KOSPI_VOLUME, self.KOSDAQ_CLOSE, self.KOSDAQ_VOLUME



# 업종명(섹터) 병합
# 각각의 섹터끼리 z스코어링을 한다.
# z = (수익률 - 평균) / 표준편차
# 각 섹터 내에서 수익률의 분포 상 위치를 순위로


In [62]:
Indicator = MultiFactorPortfolio(
    path_2022_1Q = "/content/drive/MyDrive/FSDATA/FS_2022_1Q.csv",
    path_2022_2Q = "/content/drive/MyDrive/FSDATA/FS_2022_2Q.csv",
    path_2022_3Q = "/content/drive/MyDrive/FSDATA/FS_2022_3Q.csv",
    path_2022_4Q = "/content/drive/MyDrive/FSDATA/FS_2022_4Q.csv",
    path_2023_1Q = "/content/drive/MyDrive/FSDATA/FS_2023_1Q.csv",

    path_KP_close = "/content/drive/MyDrive/FSDATA/KOSPI_CLOSE.csv",
    path_KP_volume = "/content/drive/MyDrive/FSDATA/KOSPI_VOLUME.csv",
    path_KDQ_close = "/content/drive/MyDrive/FSDATA/KOSDAQ_CLOSE.csv",
    path_KDQ_volume = "/content/drive/MyDrive/FSDATA/KOSDAQ_VOLUME.csv"
)

In [63]:
FS_2022_1Q, FS_2022_2Q, FS_2022_3Q, FS_2022_4Q, FS_2023_1Q = Indicator.loadFinancialSheet()

In [64]:
FS_2023_1Q.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2259 entries, AJ네트웍스 to 힘스
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   자산               2256 non-null   object 
 1   부채               2257 non-null   object 
 2   자본               2250 non-null   object 
 3   매출채권             1004 non-null   object 
 4   매입채무             704 non-null    object 
 5   유동자산             2258 non-null   object 
 6   유동부채             2258 non-null   object 
 7   현금성자산            2256 non-null   object 
 8   장기차입금            1062 non-null   object 
 9   총차입금             1429 non-null   object 
 10  매출액              1991 non-null   object 
 11  매출원가             1885 non-null   object 
 12  매출총이익            1878 non-null   object 
 13  판매비와관리비          1894 non-null   object 
 14  영업이익(손실)         2065 non-null   object 
 15  법인세비용차감전순이익(손실)  2066 non-null   object 
 16  당기순이익(손실)        2252 non-null   object 
 17  주당손익(EPS)       

In [65]:
Indicator.getStockList(base_date = "2022-12-31")
Indicator.getStockSector(sector_code_list = ["G10","G15","G20","G25", "G30", "G35", "G40", "G45","G50", "G55"])

In [66]:
KOSPI_TICKER, KOSDAQ_TICKER = Indicator.getFundamental(base_date = "2022-12-29")

In [78]:
pd.set_option("display.max_row", 4)
KOSPI_TICKER
# ["종목명"].isnull().sum()

,종목코드,종목명,BPS,PER,PBR,EPS,DIV,DPS,TICKER,SEC_NM_KOR
0,095570,AJ네트웍스,8075.0,3.35,0.71,1707.0,4.72,270.0,095570,산업재
1,006840,AK홀딩스,45961.0,0.00,0.37,0.0,1.16,200.0,006840,소재
...,...,...,...,...,...,...,...,...,...,...
948,403550,NaN,5241.0,73.23,4.15,297.0,0.00,0.0,403550,산업재
949,446070,NaN,0.0,0.00,0.00,0.0,0.00,0.0,446070,산업재


In [79]:
KOSDAQ_TICKER

,종목코드,종목명,BPS,PER,PBR,EPS,DIV,DPS,TICKER,SEC_NM_KOR
0,060310,3S,829.0,63.48,2.53,33.0,0.00,0.0,060310,산업재
1,054620,APS,10088.0,13.36,1.56,1179.0,0.63,100.0,054620,IT
...,...,...,...,...,...,...,...,...,...,...
1650,436610,NaN,0.0,0.00,0.00,0.0,0.00,0.0,436610,금융
1651,430460,NaN,0.0,0.00,0.00,0.0,0.00,0.0,430460,금융


In [80]:
KOSPI_CLOSE, KOSPI_VOLUME, KOSDAQ_CLOSE, KOSDAQ_VOLUME = Indicator.getMarketPQ(start_period = "2022-01-01", end_period = "2022-12-31")

In [81]:
KOSPI_CLOSE

,Date,095570,006840,027410,282330,138930,001460,001465,001040,079160,...,000547,000545,003280,37550L,373220,108320,377740,126720,403550,446070
0,2022-01-03,5320,19550,5310,143500,8390,456000,183000,84000,25150,...,26100.0,7990,2725,NaN,NaN,167800,NaN,NaN,NaN,NaN
1,2022-01-04,5320,19750,5290,142000,8470,510000,198500,84000,24850,...,26950.0,8200,2690,NaN,NaN,161900,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,2023-07-14,4300,20050,3685,173400,6780,418000,141600,62500,9300,...,3180.0,5490,2615,22750.0,542000.0,116900,5490.0,22050.0,14300.0,7890.0
379,2023-07-17,4350,20550,3710,171000,6780,427500,140200,63300,9640,...,NaN,5410,2605,23000.0,538000.0,117300,5490.0,22050.0,14390.0,8160.0


In [82]:
KOSPI_VOLUME

,Date,095570,006840,027410,282330,138930,001460,001465,001040,079160,...,000547,000545,003280,37550L,373220,108320,377740,126720,403550,446070
0,2022-01-03,5320,19550,5310,143500,8390,456000,183000,84000,25150,...,26100.0,7990,2725,NaN,NaN,167800,NaN,NaN,NaN,NaN
1,2022-01-04,5320,19750,5290,142000,8470,510000,198500,84000,24850,...,26950.0,8200,2690,NaN,NaN,161900,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,2023-07-14,4300,20050,3685,173400,6780,418000,141600,62500,9300,...,3180.0,5490,2615,22750.0,542000.0,116900,5490.0,22050.0,14300.0,7890.0
379,2023-07-17,4350,20550,3710,171000,6780,427500,140200,63300,9640,...,NaN,5410,2605,23000.0,538000.0,117300,5490.0,22050.0,14390.0,8160.0


In [83]:
KOSDAQ_CLOSE

,Date,060310,054620,265520,211270,032790,013720,035760,051500,058820,...,291810,417180,406760,418170,427950,430230,435620,409570,436610,430460
0,2022-01-03,3300,14000,25150,14850,5700,8740,139800,29400,3860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-04,3315,14200,25400,14800,5700,8870,139200,28950,3820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,2023-07-14,2855,9180,21200,12720,2920,2330,67300,28800,2150,...,6770.0,4670.0,2215.0,2200.0,2360.0,2200.0,9760.0,3410.0,2790.0,2140.0
379,2023-07-17,2825,9100,21250,12610,2895,1918,66700,28400,2285,...,7350.0,4790.0,2210.0,2200.0,2360.0,2205.0,9760.0,3520.0,2790.0,2150.0


In [84]:
KOSDAQ_VOLUME

,Date,060310,054620,265520,211270,032790,013720,035760,051500,058820,...,291810,417180,406760,418170,427950,430230,435620,409570,436610,430460
0,2022-01-03,3300,14000,25150,14850,5700,8740,139800,29400,3860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-04,3315,14200,25400,14800,5700,8870,139200,28950,3820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,2023-07-14,2855,9180,21200,12720,2920,2330,67300,28800,2150,...,6770.0,4670.0,2215.0,2200.0,2360.0,2200.0,9760.0,3410.0,2790.0,2140.0
379,2023-07-17,2825,9100,21250,12610,2895,1918,66700,28400,2285,...,7350.0,4790.0,2210.0,2200.0,2360.0,2205.0,9760.0,3520.0,2790.0,2150.0


```
Counter({'에너지': 35,
         '소재': 240,
         '산업재': 368,
         '경기관련소비재': 363,
         '필수소비재': 105,
         '건강관리': 316,
         '금융': 177,
         'IT': 645,
         '커뮤니케이션서비스': 124,
         '유틸리티': 17})
```

In [ ]:
class MultiFactorPortfolio():

  def __init__(self, main_path,
               value_factor = None, quality_factor = None, size_factor = None, divend_factor = None,
               momentum_factor = None, growth_factor = None, volatility_factor = None, flow_factor = None):

    self.MAIN_DATA = pd.read_csv(main_path, index_col = False, encoding = "utf-8")
    self.value_factor = value_factor
      # FN Guide 모멘텀 & 밸류 지수(kBSTAR 모멘텀 밸류 ETF의 BM)
    self.quality_factor = quality_factor
      # 종목의 이익의 질
      # 제임스 오쇼너시의 데이터 기반 접근법 전략
      # ROC > 0.13 & P/E < 20, P/B < 2, 부채 대 자본비율 > 1.5 or 배당 수익률 > 0.02, EPS 연성장률 > 0.2, 배당 수익률 > 0.04 or 부채-자본비율 < 1, 2개월 가격 모멘텀 > 0 그리고 EPS 연성장률 > 0.20
    self.divend_factor = divend_factor
      # 배당
    self.size_factor = size_factor
      # 시가총액과 자산규모
    self.momentum_factor = momentum_factor
      # 시장민감도
      # Momentum Factor
      # PER < 15, ROIC > 0.15, 부채 대비 자본비율 < 0.5, 총마진성장률 > 0.03, FCF > 시가총액의 0.05
    self.growth_factor = growth_factor
      # KP + KDQ 보통주(시가총액 7,000억 원, 60일 평균 거래대금 상위 20% 이내) 종목 중
      # 1) 현재 영업이익이 적자이거나 역성장하며 부진했으나,
      # 2) 향후 흑자전환 등으로 분기 영업이익이 개선될 것으로 크게 기대되는 기업
      # 3) 2024년 이익성장률 상위 & 이익전망 하향이 크지 않은 종목 -> 단, 이경우 어닝서프라이즈 데이터가 필요함(컨센서)
    self.volatility_factor = volatility_factor
      # 종목의 변동성
    self.flow_factor = flow_factor
      # 종목의 수급, 거래량, 거래대금


  def FactorDescriptor(self):
    # 종목 수익에 영향을 미치는 요인 선정
    # 1.1) Descriptor Selection(특성치 선정) : 주식은 스타일 팩터와 업종 팩터로 구부
    # 1.2) Wubsorization(극단치 또는 이상치 처리)
    # 1.3) Feature Engineering
    # 1.4) Weighting(특성치별 가중치, PCA)
    # 1.5) Rescaling Zscore(특성치 조합에 따른 Factor의 표준편차 증감을 재조정)


    # 1.1).(1) 가치투자 팩터 인덱스는 액티브 투자전략이라 매년 상당한 교체가 발생하고, 성장에 대한 시대의 기대가 낮은 종목으로 교체된다(EPS상승률이 높은 이유)
    # sales | EBITDA / sales | Earnings / EBITDA

    # EV/EBITDA | F-score(ROA > 0, CFO > 0, dROA > 0, CFO > ROA, dLEVERAGE < 0, dLIQUID > 0, EQ_OFFER(발행주식 수), dMARGIN(매출총이익) > 0, dTURNOVER(회전율) > 0)
    # Value와 Glamour전략

    # [수익성]
    self.MAIN_DATA["ROA"] = self.MAIN_DATA["지배주주순이익"] / self.MAIN_DATA["자산"]
    self.MAIN_DATA["CFO"] = self.MAIN_DATA["영업활동으로인한현금흐름"] / self.MAIN_DATA["자산"]
    self.MAIN_DATA["ACCURUAL"] = self.MAIN_DATA["CFO"] - self.MAIN_DATA["ROA"]
    # [재무적 성과]
    self.MAIN_DATA["LEVER"] = self.MAIN_DATA["장기차입금"] / self.MAIN_DATA["자산"]
    self.MAIN_DATA["LIQ"] = self.MAIN_DATA["유동자산"] / self.MAIN_DATA["유동부채"]
    self.MAIN_DATA["OFFER"] = self.MAIN_DATA["유상증자"]
    # [운영 효율성]
    self.MAIN_DATA["MARGIN"] = self.MAIN_DATA["매출총이익"] / self.MAIN_DATA["매출액"]
    self.MAIN_DATA["TURNOVER"] = self.MAIN_DATA["매출액"] / self.MAIN_DATA["자산"]
    # [F-Score] 고평가주
    for subject in ["ROA", "CFO", "ACCURUAL", "LEVER", "LIQ", "OFFER", "MARGIN", "TURNOVER"]:
      self.MAIN_DATA["F-SCORE"] = self.MAIN_DATA["F-SCORE"] + self.MAIN_DATA[subject].apply(lambda data: np.where(data > 0, 1, 0))

    # [value] EV / EBITDA = 기업의 시장가치 / 세전영업이익(순이익, 이자비용, 법인세, 감가상각비 차감 전) = (시가총액 + 총차입금 - 현금성 자산) / (영업이익 + 감가상각비 + 제세금)



    # 1.1).(2) 성장성 팩터
    self.MAIN_DATA['영업이익 증가율'] = (self.MAIN_DATA['영업이익'].diff() / self.MAIN_DATA['영업이익'].shift(1)).fillna(0) * 100
    self.MAIN_DATA['매출액 증가율'] = (self.MAIN_DATA['매출액'].diff() / self.MAIN_DATA['매출액'].shift(1)).fillna(0) * 100
    self.MAIN_DATA['당기순이익 증가율'] = (self.MAIN_DATA['당기순이익'].diff() / self.MAIN_DATA['당기순이익'].shift(1)).fillna(0) * 100

    # 영업이익, 매출액, 당기순이익의 전환지표
    for i in range(len(status)):
        self.MAIN_DATA[status[i]] = np.nan
        self.MAIN_DATA.loc[
            (self.MAIN_DATA[three_indicators[i]] > 0) & (self.MAIN_DATA[three_indicators[i]].shift(-1) <= 0), status[i]] = "흑자 전환"


    # 1.1).(3) 모멘텀[시장민감도] 팩터
    # t-2월부터 t-12월까지 11개월간 누적수익률을 연간 수익률로 환산한 값

    self.MAIN_DATA["momentum"] = self.MAIN_DATA["return"].pct_change(12)
      # return = 12개월(252일) 수익률
    self.MAIN_DATA["momentum"] = (
        ( self.MAIN_DATA["cummulative_return"].shift(-1) / (self.MAIN_DATA["cummulative_return"].shift(-12) )
        ) -1).groupby(level = [ticker_feature_name] )
      # the first mom signal has to start 12 months after the start of date
    self.MAIN_DATA["momentum"] = self.MAIN_DATA.groupby(level = [ticker_feature_name])["return"].apply(lambda x: x.pct_change(11))


    # 1.1).(4) 사이즈[시가총액 = 보통주 종가*발행주식 수] 팩터
    # cf.) 장부가치 대비 시장가치 = 시가총액 / (자본총계 - 우선주자본금)
    #




  # def FactorExposure(self):
    # 유사한 특성치를 요인으로 그룹핑
    # 전략이 보유하고 있는 포트폴리오의 팩터 스타일 점수를 기준일자 바탕으로 제공하여 해당 전략의 특성을 한눈에 확인가


  # def Risk&ReturnManagement(self):
    # 포트폴리오 및 벤치마크 대비 변동성 예측 / 수익률의 원천인 요인별로 수익률 분해


```
# MaxDrawDown()
  # 최대손실률 = 일정기간(window_length)동안 최고점과 최저점 값의 차이를 백분율로 나타낸 리스크 관련 지표
  # MDD가 높을 수록 변동성이 높다고 판단한다.
  def MaxDrawDown(self):
  # 1. 피크(최고점) 식별
    drawdowns = fmax.accumulate(self.Market_sequrities_data["종가"], axis=0) - self.Market_sequrities_data["종가"]
    drawdowns[isnan(drawdowns)] = NINF
    drawdown_ends = nanargmax(drawdowns, axis=0)
  # 2. 최저점 식별
  # 3. 하락률 계산 = (최저값 - 최고값) / 최고값
    for i, end in enumerate(drawdown_ends):
      peak = nanmax(self.Market_sequrities_data["종가"][: end + 1, i])
      out[i] = (peak - self.Market_sequrities_data["종가"][end, i]) / self.Market_sequrities_data["종가"][end, i]
```

In [ ]:
class FactorEvaluation_PrincipalComponentAnalysis():
  # 요인의 수익 기여도

  def __init__(self, file_path: str):
    self.differencing_market_return = pd.read_csv(file_path, index_col = 0)


# linearCombinationPCA()
  def linearCombinationPCA(self, k: int):
  # 1. 표준화
    mean_matrix = self.differencing_market_return.mean()
    std_matrix = self.differencing_market_return.std()
    substract_mean_matrix = (self.differencing_market_return - mean_matrix)
    normalized_matrix = (self.differencing_market_return - mean_matrix) / std_matrix

  # 2. 공분산 행렬(Calculate the Covariance Matrix of the mean-centered data.)
    covariance_matrix = np.cov(substract_mean_matrix.T)
    # 공분산 행렬의 고유 분해(고유값과 고유벡터를 찾는 작업)를 수행하여 고유벡터(분산방향)와 고유값
    # 행렬 A의 고유값은 A−λI의 행렬식이 0이 되도록 하는 특성방정식(characteristic equation)의 해를 구하면 된다.
    # 고윳값을 알면 다음 연립 방정식을 풀어 고유벡터를 구할 수 있다. (A−λI)v=0
    eigen_values, eigen_vectors = np.linalg.eigh(covariance_matrix)

  # 3. 고윳값과 고유벡터를 내림차순 정렬(모든 고유벡터는 서로 직교함)
  # numpy.linalg.eigh( M, UPLO = 'L'/'U'/'None'(optional))
  # [M] Matrix, M행렬은 본질적으로 실제 대칭인 입력
  # [UPLO] 행렬의 고유값이 계산되는 위(U) 또는 아래(L) 삼각형 부분
    sorted_index = np.arange(0, len(eigen_values), 1)
    sorted_index = ([x for _, x
                     in sorted(zip(eigen_values, sorted_index))])[::-1]
    # 동일한 기능을 하는 소스코드
    # sorted_index = np.argsort(eigen_values)[::-1]
      # [np.argsort] returns an array of indices of the same shape.

    sorted_eigen_values = eigen_values[sorted_index]
    sorted_eigen_vectors = eigen_vectors[:, sorted_index]
    print("내림차순 정렬한 고유값",  "\n", sorted_eigen_values,
          "\n", "내림차순 정렬한 고유벡터",  "\n", sorted_eigen_vectors)

  # 4. 원하는 결합정도에 따라 상위 k의 고유벡터를 선택하는데 모든 고유값의 누적합계로  설명된 분산 기여율 계산
    sum_eigen_values = np.sum(sorted_eigen_values)
    explained_variance = sorted_eigen_values / sum_eigen_values
    cummulative_variance = np.cumsum(explained_variance)
    print("누적 분산 기여율", "\n", cummulative_variance, "\n")
      # [0.92443721 0.9931338  0.99939027 0.99976834 1.        ]

  # 5. 변환된 데이터로 고유벡터의 전치행렬의 내적을 취하여 원본 데이터를 결합(데이터를 중앙에 배치하기 위해 평균을 뺌)
    n_components = k
      # k = 1
    eigen_vectors_subset = sorted_eigen_vectors[:, 0:n_components]

  # 6. 고유벡터를 사용하여 데이터의 좌표방향변환(회전) -> 정사영 / 투영 / 내적
  # PC(주성분 공간에서의 좌표) = Z(표준화된 원래의 좌표)*A(좌표변환행렬) = [z_x, z_y]([a_1, a_2]).T = a_1*z_x + a_2*z_y
  # A(시계방향 45도 회전행렬, Rotation Matrix) = [(cos(45), -sin(45)), (sin(45), cos(45))] = [(0.71, -0.71), (0.71, 0.71)]
    pca_matrix = np.dot(substract_mean_matrix, eigen_vectors_subset)
    # print("좌표변환한 행렬", "\n", pca_matrix, "\n")

    reduction_matrix = pd.Series(np.array(pca_matrix, ndmin = 1).tolist())
    for parenthesis in ["[", "]"]:
      reduction_matrix = reduction_matrix.apply(lambda data: str(data).replace(parenthesis, ""))
    reduction_matrix = reduction_matrix.astype("float")
    reduction_matrix.index = self.differencing_market_index.index
    return reduction_matrix

    # 에러코드
      # ValueError: shapes (5,1) and (5,) not aligned: 1 (dim 1) != 5 (dim 0)
      # k_reduction_matrix  = np.dot(eigen_vectors_subset, mean_matrix.transpose()).transpose()


# SklearnlinearPCA()
# Scikit-Learn 모듈을 활용한 주성분과 결과가 같은지 비교
  def SklearnlinearPCA(self, k: int):

    from sklearn.decomposition import PCA

    sklearn_pca= PCA(n_components = k)
    self.scaled_market_return = self.differencing_market_return.apply(
        lambda data: data - data.mean() / data.std())

    sklearn_pca.fit(self.scaled_market_return)
    sklearn_pca_matrix = sklearn_pca.transform(self.scaled_market_return)
    return sklearn_pca_matrix


# sparsePCA() 희소주성분분석
  def SklearnSparsePCA(self, k: int, c: float):

    from sklearn.decomposition import SparsePCA
    # L1 정규화는 가중치 벡터의 절댓값 합을 최소화
    # L1 정규화를 PCA에 적용함으로써, 주성분 벡터의 일부 요소는 0으로 만들어지고, 다른 요소들은 비교적 큰 값을 가지게 된다.
    # L1 정규화의 강도를 조절하는 하이퍼파라미터
    sklearn_sparse_pca= SparsePCA(n_components = k, alpha = c , random_state = 2023)
    self.scaled_market_return = self.differencing_market_return.apply(
        lambda data: data - data.mean() / data.std())

    sklearn_sparse_pca.fit(self.scaled_market_return)
    sklearn_sparse_pca_matrix = sklearn_sparse_pca.transform(self.scaled_market_return)

    sparse_pca_matrix = pd.Series(np.array(sklearn_sparse_pca_matrix, ndmin = 1).tolist())
    for parenthesis in ["[", "]"]:
      sparse_pca_matrix = sparse_pca_matrix.apply(lambda data: str(data).replace(parenthesis, ""))
    sparse_pca_matrix = sparse_pca_matrix.astype("float")
    sparse_pca_matrix.index = self.differencing_market_index.return
    return sparse_pca_matrix


In [ ]:
PCAinstance = FactorEvaluation_PrincipalComponentAnalysis(
    file_path =  "/content/drive/MyDrive/")

In [ ]:
factorReturn_linearPCA = PCAinstance.linearCombinationPCA(k = 1)
# factorIndex_sklearnSparsePCA = PCAinstance.SklearnSparsePCA(k = 1, c = 0.1)

In [ ]:
factorReturn_linearPCA